# Tensorflow MNIST Classifier demo

>⚠️ **Warning:** Some of the attacks in this demo, _deepfool_ and _CW_ in particular, are computationally expensive and will take a very long to complete if run using the CPUs found in a typical personal computer.
> For this reason, it is highly recommended that you run these demos on a CUDA-compatible GPU.

This notebook contains an end-to-end demostration of Dioptra that can be run on any modern laptop.
Please see the [example README](README.md) for instructions on how to prepare your environment for running this example.

## Setup

Below we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected,

In [2]:
# Import packages from the Python standard library
import importlib.util
import os
import sys
import pprint
import time
import warnings
from pathlib import Path


def register_python_source_file(module_name: str, filepath: Path) -> None:
    """Import a source file directly.

    Args:
        module_name: The module name to associate with the imported source file.
        filepath: The path to the source file.

    Notes:
        Adapted from the following implementation in the Python documentation:
        https://docs.python.org/3/library/importlib.html#importing-a-source-file-directly
    """
    spec = importlib.util.spec_from_file_location(module_name, str(filepath))
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)


# Filter out warning messages
warnings.filterwarnings("ignore")

# Experiment name
EXPERIMENT_NAME = "jminiter_feature_squeezing"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = "http://localhost:80"

# Set DIOPTRA_RESTAPI_URI variable if not defined, used to connect to RESTful API service
if os.getenv("DIOPTRA_RESTAPI_URI") is None:
    os.environ["DIOPTRA_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Register the examples/scripts directory as a Python module
register_python_source_file("scripts", Path("..", "scripts", "__init__.py"))

from scripts.client import DioptraClient
from scripts.utils import make_tar

# Import third-party Python packages
import numpy as np
from mlflow.tracking import MlflowClient

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

In [3]:
mnist_model = "mnist_le_net" 
mnist_shallow = "mnist_shallow_net"
model_id = 5
model_id_shallow = 3
mnist_dataset = "/dioptra/data/Mnist"
mlflow_queue = "tensorflow_gpu"

## Dataset

We obtained a copy of the MNIST dataset when we ran `download_data.py` script. If you have not done so already, see [How to Obtain Common Datasets](https://pages.nist.gov/dioptra/getting-started/acquiring-datasets.html).
The training and testing images for the MNIST dataset are stored within the `/dioptra/data/Mnist` directory as PNG files that are organized into the following folder structure,

    Mnist
    ├── testing
    │   ├── 0
    │   ├── 1
    │   ├── 2
    │   ├── 3
    │   ├── 4
    │   ├── 5
    │   ├── 6
    │   ├── 7
    │   ├── 8
    │   └── 9
    └── training
        ├── 0
        ├── 1
        ├── 2
        ├── 3
        ├── 4
        ├── 5
        ├── 6
        ├── 7
        ├── 8
        └── 9

The subfolders under `training/` and `testing/` are the classification labels for the images in the dataset.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `src/MLproject` file.
To run these entrypoints within Dioptra's architecture, we need to package those files up into an archive and submit it to the Dioptra RESTful API to create a new job.
For convenience, we provide the `make_tar` helper function defined in `examples/scripts/utils.py`.

In [32]:
make_tar(["src"], WORKFLOWS_TAR_GZ)

PosixPath('/c/users/jtsexton/documents/github/dioptra/examples/tensorflow-mnist-feature-squeezing/workflows.tar.gz')

To connect with the endpoint, we will use a client class defined in the `examples/scripts/client.py` file that is able to connect with the Dioptra RESTful API using the HTTP protocol.
We connect using the client below.
The client uses the environment variable `DIOPTRA_RESTAPI_URI`, which we configured at the top of the notebook, to figure out how to connect to the Dioptra RESTful API.

In [18]:
restapi_client = DioptraClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment named `"mnist"` exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [19]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'lastModified': '2023-05-01T17:30:11.663914',
 'createdOn': '2023-05-01T17:30:11.663914',
 'experimentId': 2,
 'name': 'jminiter_feature_squeezing'}

Next, we need to train our model.
Depending on the specs of your computer, training either the shallow net model or the LeNet-5 model on a CPU can take 10-20 minutes or longer to complete.
If you are fortunate enough to have access to a dedicated GPU, then the training time will be much shorter.

So that we do not start this code by accident, we are embedding the code in a text block instead of keeping it in an executable code block.
**If you need to train one of the models, create a new code block and copy and paste the code into it.**

```python
# Submit training job for the shallow network architecture
response_shallow_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P model_architecture=shallow_net",
        "-P epochs=30",
        "-P register_model_name=mnist_shallow_net",
    ]),
    queue=mlflow_queue,
)

print("Training job for shallow neural network submitted")
print("")
pprint.pprint(response_shallow_train)
```

```python
# Submit training job for the LeNet-5 network architecture
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join([
        "-P model_architecture=le_net",
        "-P epochs=30",
        "-P register_model_name=mnist_le_net",
    ]),
    queue=mlflow_queue,
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)
```

In [32]:
response_shallow_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join(
        [
            "-P model_architecture=shallow_net",
            "-P epochs=30",
            "-P register_model_name=mnist_shallow_net",
            f"-P data_dir={mnist_dataset}",
        ]
    ),
    queue=mlflow_queue,
)

print("Training job for shallow neural network submitted")
print("")
pprint.pprint(response_shallow_train)

# Submit training job for the LeNet-5 network architecture
response_le_net_train = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="train",
    entry_point_kwargs=" ".join(
        [
            "-P model_architecture=le_net",
            "-P epochs=30",
            "-P register_model_name=mnist_le_net",
            f"-P data_dir={mnist_dataset}",
        ]
    ),
    queue=mlflow_queue,
)

print("Training job for LeNet-5 neural network submitted")
print("")
pprint.pprint(response_le_net_train)

Training job for shallow neural network submitted

{'createdOn': '2023-05-01T18:40:32.957998',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P model_architecture=shallow_net -P epochs=30 -P '
                     'register_model_name=mnist_shallow_net -P '
                     'data_dir=/dioptra/data/Mnist',
 'experimentId': 2,
 'jobId': '8aa0249f-d335-457a-83ca-3daa2873ca4b',
 'lastModified': '2023-05-01T18:40:32.957998',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/2d225457d65b452192fde473aef3bd30/workflows.tar.gz'}
Training job for LeNet-5 neural network submitted

{'createdOn': '2023-05-01T18:40:33.109818',
 'dependsOn': None,
 'entryPoint': 'train',
 'entryPointKwargs': '-P model_architecture=le_net -P epochs=30 -P '
                     'register_model_name=mnist_le_net -P '
                     'data_dir=/dioptra/data/Mnist',
 'experimentId': 2,
 'jobId': '56cb4d7b-74a1-49c2-802a-b14822158cab',
 

In [20]:
restapi_client.delete_custom_task_plugin(name = "feature_squeezing")

{'collection': 'dioptra_custom',
 'status': 'Success',
 'taskPluginName': ['feature_squeezing']}

In [21]:
%%bash

python ../scripts/register_task_plugins.py --plugins-dir ../task-plugins

╭─────────────────────────────────────────────────╮
│ Dioptra Examples - Register Custom Task Plugins │
╰─────────────────────────────────────────────────╯
 ‣ plugins_dir: ../task-plugins
 ‣ api_url: http://localhost
Ⓘ  Skipped. The custom task plugin 'evaluation' is already registered.
 ✔ Success! Registered the custom task plugin 'feature_squeezing'.
 ✔ Custom task plugin registration is complete.


The following code block generates adversarial images on the MNIST dataset using the Fast Gradient Method attack and then attempts to classify the adversarial images.

| parameter | type | description |
| --- | --- | --- |
| `eps` | _float_ | Attack step size. \[default: 0.3\] |
| `eps_step` | _float_ | Step size of input variation for minimal perturbation computation. [default: 0.1] |
| `targeted` | _bool_ | Indicates whether the attack is targeted (True) or untargeted (False). [default: False] |
| `num_random_init` | _int_ | Number of random initializations within the epsilon ball. For ``random_init=0`` starting at the original input. [default: 0] |
| `minimal` | _bool_ | Indicates if computing the minimal perturbation (True). If True, also define eps_step for the step size and eps for the maximum perturbation. [default: False] |

In [34]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="fgm",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_model}",
            f"-P data_dir={mnist_dataset}",
            f"-P model_version={model_id}"
        ]
    ),
    queue = mlflow_queue,
    timeout="1m"

)

print("FGM attack job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"]) 
    
response_infer = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model_version={model_id}",
            f"-P model_name={mnist_model}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_fgm_le_net["jobId"]
)
    
print("Dependent jobs submitted")

FGM attack job submitted

{'createdOn': '2023-05-01T18:58:01.568696',
 'dependsOn': None,
 'entryPoint': 'fgm',
 'entryPointKwargs': '-P model_name=mnist_le_net -P '
                     'data_dir=/dioptra/data/Mnist -P model_version=5',
 'experimentId': 2,
 'jobId': 'edcf5b95-6412-4702-b5ff-2dc63d257ea5',
 'lastModified': '2023-05-01T18:58:01.568696',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1m',
 'workflowUri': 's3://workflow/cc58dc7387184dbb97e2d7b6afcdaa53/workflows.tar.gz'}

Dependent jobs submitted


This block does the same as the previous block, but applies the feature squeezing defense between the attack and infer steps.
This pre-processing defense compresses the images being classified by our neural network such that their color depth is reduced to a binary, monochrome pallete.
The level of compression can be tuned by adjusting the bit_depth parameter below (use values between 1 (binary) and 8 (original image color depth) to tune the defense.

**FGM parameters**

| parameter | type | description |
| --- | --- | --- |
| `eps` | _float_ | Attack step size. \[default: 0.3\] |
| `eps_step` | _float_ | Step size of input variation for minimal perturbation computation. [default: 0.1] |
| `targeted` | _bool_ | Indicates whether the attack is targeted (True) or untargeted (False). [default: False] |
| `num_random_init` | _int_ | Number of random initializations within the epsilon ball. For ``random_init=0`` starting at the original input. [default: 0] |
| `minimal` | _bool_ | Indicates if computing the minimal perturbation (True). If True, also define eps_step for the step size and eps for the maximum perturbation. [default: False] |

**Feature squeezing parameters**

| parameter | type | description |
| --- | --- | --- |
| `bit_depth` | _int_ | An integer between 1-8 that defines the color depth of the squeezed image. [default: 8] |

In [17]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="fgm",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_model}",
            f"-P data_dir={mnist_dataset}",
            f"-P model_version={model_id}"
        ]

    ),
    queue = mlflow_queue,

)

print("FGM attack  job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])
print(response_fgm_le_net)

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model={mnist_model}/{model_id}",
            "-P model_architecture=le_net",
            "-P bit_depth=1",
            "-P batch_size=32"
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_fgm_le_net["jobId"],
    timeout = "1h"
)

while mlflow_run_id_is_not_known(response_feature_squeeze):
    time.sleep(1)
    response_feature_squeeze = restapi_client.get_job_by_id(response_feature_squeeze["jobId"]) 
print(response_feature_squeeze)
response_infer_defended = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model_version={model_id}",
            f"-P model_name={mnist_model}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_feature_squeeze["jobId"]
)
    
print("Dependent jobs submitted")

FGM attack  job submitted

{'createdOn': '2023-05-02T15:43:53.677153',
 'dependsOn': None,
 'entryPoint': 'fgm',
 'entryPointKwargs': '-P model_name=mnist_le_net -P '
                     'data_dir=/dioptra/data/Mnist -P model_version=5',
 'experimentId': 2,
 'jobId': '54f4f35b-2893-4396-8b7a-4aad671fbb11',
 'lastModified': '2023-05-02T15:43:53.677153',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/d03615c553304117adf01b2956c860fa/workflows.tar.gz'}

{'experimentId': 2, 'entryPoint': 'fgm', 'createdOn': '2023-05-02T15:43:53.677153', 'mlflowRunId': 'd7ef0a1331384dc597b53d997f349e41', 'workflowUri': 's3://workflow/d03615c553304117adf01b2956c860fa/workflows.tar.gz', 'lastModified': '2023-05-02T15:44:20.748660', 'dependsOn': None, 'status': 'queued', 'timeout': '24h', 'entryPointKwargs': '-P model_name=mnist_le_net -P data_dir=/dioptra/data/Mnist -P model_version=5', 'jobId': '54f4f35b-2893-4396-8b7a-4aad671fbb11', 'queueId': 2}

This block uses the carlini-wagner attack with the Linf distance metric to generate adversarial images and checks the model's accuracy against the attack.

**Carlini Wagner Parameters**

| parameter | type | description |
| --- | --- | --- |
| `targeted` | _bool_ | Indicates whether the attack is targeted (True) or untargeted (False). [default: False] |
| `learning_rate` | _float_ | The initial learning rate for the attack algorithm. Smaller values produce better results but are slower to converge. [default: 0.01] |

In [16]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="cw_inf",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_shallow}",
            f"-P model_version={model_id_shallow}",
            "-P model_architecture=shallow_net",
            "-P targeted=True",
            "-P max_iter=10",
            "-P confidence=0.0",
            f"-P data_dir={mnist_dataset}",
            "-P learning_rate=0.01",
            "-P verbose=True",
            "-P batch_size=32"
        ]

    ),
    queue = mlflow_queue,

)

print("Carlini Wagner attack job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
         [
             f"-P run_id={response_fgm_le_net['mlflowRunId']}",
             f"-P model_version={model_id_shallow}",
             f"-P model_name={mnist_shallow}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_fgm_le_net["jobId"],
)

print("Dependent jobs submitted")

Carlini Wagner attack job submitted

{'createdOn': '2023-05-02T16:45:24.933323',
 'dependsOn': None,
 'entryPoint': 'cw_inf',
 'entryPointKwargs': '-P model_name=mnist_shallow_net -P model_version=3 -P '
                     'model_architecture=shallow_net -P targeted=True -P '
                     'max_iter=10 -P confidence=0.0 -P '
                     'data_dir=/dioptra/data/Mnist -P learning_rate=0.01 -P '
                     'verbose=True -P batch_size=32',
 'experimentId': 2,
 'jobId': '751b009c-c25c-414b-bfe9-8bc7991420d9',
 'lastModified': '2023-05-02T16:45:24.933323',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/473dcd7321d04b5888ff6a63a6f27df1/workflows.tar.gz'}

Dependent jobs submitted


This block does the same as the previous block, but applies the feature squeezing defense between the attack and infer steps.

In [33]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="cw_inf",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_shallow}", 
            f"-P model_version={model_id_shallow}", 
            "-P model_architecture=shallow_net",
            "-P targeted=True", 
            "-P max_iter=20",
            "-P confidence=0.0", 
            f"-P data_dir={mnist_dataset}", 
            "-P learning_rate=0.01", 
            "-P verbose=True", 
            "-P batch_size=32"
        ]

    ),
    queue = mlflow_queue,
)

print("Carlini Wagner attack job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model={mnist_shallow}/{model_id_shallow}",
            "-P model_architecture=shallow_net",
            "-P bit_depth=1",
            "-P batch_size=32"
        ]
    ),
    depends_on=response_fgm_le_net["jobId"],
    queue = mlflow_queue,
)

while mlflow_run_id_is_not_known(response_feature_squeeze):
    time.sleep(1)
    response_feature_squeeze = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model_name={mnist_shallow}",
            f"-P model_version={model_id_shallow}"
        ]
    ),
    depends_on=response_feature_squeeze["jobId"],
    queue = mlflow_queue,
)

print("Dependent jobs submitted")

Carlini Wagner attack job submitted

{'createdOn': '2023-05-02T18:31:15.296745',
 'dependsOn': None,
 'entryPoint': 'cw_inf',
 'entryPointKwargs': '-P model_name=mnist_shallow_net -P model_version=3 -P '
                     'model_architecture=shallow_net -P targeted=True -P '
                     'max_iter=20 -P confidence=0.0 -P '
                     'data_dir=/dioptra/data/Mnist -P learning_rate=0.01 -P '
                     'verbose=True -P batch_size=32',
 'experimentId': 2,
 'jobId': '076dd10a-a195-46ae-b7bb-812c7f23f901',
 'lastModified': '2023-05-02T18:31:15.296745',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/db13160548664e17ac946c8f53a01b4a/workflows.tar.gz'}

Dependent jobs submitted


This block uses the carlini-wagner attack using the L2 distance metric to generate adversarial images and checks the model's accuracy against the attack.

In [35]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="cw_l2",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_shallow}",
            f"-P model_version={model_id_shallow}",
            "-P binary_search_steps=50", 
            "-P initial_const=0.01",
            "-P model_architecture=shallow_net",
            "-P max_iter=10",
            f"-P data_dir={mnist_dataset}",
            "-P verbose=True",
            "-P batch_size=32"]

    ),
    queue = mlflow_queue,

)

print("Carlini Wagner attack job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}",
        ]
    ),
    queue = mlflow_queue,

    depends_on=response_fgm_le_net["jobId"],
)

print("Dependent jobs submitted")


Carlini Wagner attack job submitted

{'createdOn': '2023-05-02T18:46:14.839536',
 'dependsOn': None,
 'entryPoint': 'cw_l2',
 'entryPointKwargs': '-P model_name=mnist_shallow_net -P model_version=3 -P '
                     'binary_search_steps=50 -P initial_const=0.01 -P '
                     'model_architecture=shallow_net -P max_iter=10 -P '
                     'data_dir=/dioptra/data/Mnist -P verbose=True -P '
                     'batch_size=32',
 'experimentId': 2,
 'jobId': 'f723253d-a578-49da-ad57-80a835848c1d',
 'lastModified': '2023-05-02T18:46:14.839536',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/c5b1a6fafb414487a91c3c4fa6ff94cb/workflows.tar.gz'}

Dependent jobs submitted


This block uses the Deepfool attack to generate adversarial MNIST images, applies the feature squeezing defense, and checks the model's accuracy against the defended adversarial dataset.

**Unique Deepfool parameters**

| parameter | type | description |
| --- | --- | --- |
| `epsilon` | _float_ | Overshoot parameter. [default: 0.00001] |
| `nb_grads` | _int_ | Number of class gradients to compute. [default: 10] |

In [ ]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deepfool",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_model}",
            f"-P data_dir={mnist_dataset}/training",
            "-P model_architecture=le_net",
            "-P batch_size=32",
            "-P max_iter=10",
            "-P verbose=True",
            "-P nb_grads=10",
            "-P epsilon=0.000001",
        ],
    ),
    queue = mlflow_queue,
)

print("Deepfool attack (Mobilenet architecture) job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model={mnist_model}/{model_id}",
            "-P model_architecture=le_net",
            "-P bit_depth=1",
            "-P batch_size=32",
            f"-P data_dir={mnist_dataset}/training",
        ],
    ),
    depends_on=response_fgm_le_net["jobId"],
    queue = mlflow_queue,
)

while mlflow_run_id_is_not_known(response_feature_squeeze):
    time.sleep(1)
    response_feature_squeeze = restapi_client.get_job_by_id(response_feature_squeeze["jobId"])

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_feature_squeeze['mlflowRunId']}",
            f"-P model_name={mnist_model}",
            "-P batch_size=32"
        ],
    ),
    depends_on=response_feature_squeeze["jobId"],
    queue = mlflow_queue,
)

print("Dependent jobs submitted")

This block does the same as the previous block, but applies the feature squeezing defense between the attack and infer steps.

In [ ]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_le_net = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="deepfool",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_model}",
            f"-P data_dir={mnist_dataset}/training",
            "-P model_architecture=le_net",
            "-P batch_size=32",
            "-P max_iter=10",
            "-P verbose=True",
            "-P nb_grads=10",
            "-P epsilon=0.000001",
            "-P image_size=28,28"
        ],
    ),
    queue = mlflow_queue,
)

print("Deepfool attack (Mobilenet architecture) job submitted")
print("")
pprint.pprint(response_fgm_le_net)
print("")

while mlflow_run_id_is_not_known(response_fgm_le_net):
    time.sleep(1)
    response_fgm_le_net = restapi_client.get_job_by_id(response_fgm_le_net["jobId"])

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_le_net['mlflowRunId']}",
            f"-P model_name={mnist_model}",
            "-P batch_size=32",
        ]
    ),
    depends_on=response_fgm_le_net["jobId"],
    queue = mlflow_queue,
)

print("Dependent jobs submitted")

This block applies the Jacobian Saliency Map Approach attack to generate adversarial images for the MNIST dataset.

**Unique JSMA parameters**

| parameter | type | description |
| --- | --- | --- |
| `theta` | _float_ | Amount of Perturbation introduced to each modified feature per step (can be positive or negative). [default: 0.1] |
| `gamma` | _float_ | Maximum fraction of features being perturbed (between 0 and 1). [default: 1.0] |

In [ ]:

def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_jsma = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="jsma",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}", 
            "-P model_architecture=le_net", 
            "-P theta=4.5",
            "-P verbose=True",
            "-P gamma=1.0", 
            f"-P data_dir={mnist_dataset}"
        ] 

    ),
    queue = mlflow_queue,

)

print("JSMA attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_jsma)
print("")


while mlflow_run_id_is_not_known(response_jsma):
    time.sleep(1)
    response_jsma = restapi_client.get_job_by_id(response_jsma["jobId"])

response_le_net_infer_le_net_fgm = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_jsma['mlflowRunId']}",
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_jsma["jobId"],
)

print("Dependent jobs submitted")

This block does the same as the previous block, but applies the feature squeezing defense between the attack and infer steps.

In [ ]:
def mlflow_run_id_is_not_known(response_fgm):
    return response_fgm["mlflowRunId"] is None and response_fgm["status"] not in [
        "failed",
        "finished",
    ]

response_jsma = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="jsma",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}", 
            "-P model_architecture=le_net", 
            "-P theta=4.5",
            "-P verbose=True",
            "-P gamma=1.0", 
            f"-P data_dir={mnist_dataset}"
        ] 
    ),
    queue = mlflow_queue,
)

print("JSMA attack (LeNet-5 architecture) job submitted")
print("")
pprint.pprint(response_jsma)
print("")

while mlflow_run_id_is_not_known(response_jsma):
    time.sleep(1)
    response_jsma = restapi_client.get_job_by_id(response_jsma["jobId"])

response_feature_squeeze = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="feature_squeeze",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_jsma['mlflowRunId']}",
            f"-P model={mnist_model}/{model_id}",
            "-P model_architecture=le_net",
            "-P bit_depth=1",
            "-P batch_size=32"
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_jsma["jobId"],
)

while mlflow_run_id_is_not_known(response_feature_squeeze):
    time.sleep(1)
    response_feature_squeeze = restapi_client.get_job_by_id(response_feature_squeeze["jobId"])

response_le_net_infer_le_net_jsma = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_jsma['mlflowRunId']}",
            f"-P model_name={mnist_model}",
            f"-P model_version={model_id}",
        ]
    ),
    queue = mlflow_queue,
    depends_on=response_feature_squeeze["jobId"],
)

print("Dependent jobs submitted")